# Biological Classification problem with phenotype

Note: I don't own the data. It is gitignored to protect patent

In [ ]:
import pandas as pd

df = pd.read_csv("plant_traits_espece_ee.csv")
df.columns = ['organs', 'trait', 'trait_code', 'finer_code/detail'] + df.columns.to_list()[4:]
df = df.transpose()
df = df.reset_index()

n = len(df.iloc[0])

import numpy as np

# p_organs, p_trait, p_trait_code = [np.nan] * 3


for j in range(5, n):
    if pd.isnull(df.iloc[0, j]):
        df.iloc[0, j] = p_organs
    else:
        p_organs = df.iloc[0, j]
    
    if pd.isnull(df.iloc[1, j]):
        df.iloc[1, j] = p_trait
    else:
        p_trait = df.iloc[1, j]

    if pd.isnull(df.iloc[2, j]):
        df.iloc[2, j] = p_trait_code
    else:
        p_trait_code = df.iloc[2, j]


    df.iloc[3, j] = ">".join(map(str, (p_organs, p_trait, p_trait_code)))

df.iloc[0:3,4] = np.nan


df2 = df.iloc[3:,:]
# df2[0] = df2[0].astype(str)
df2.iloc[0,0] = 'organs>trait>trait_code>detail'
new_header = df2.iloc[0]
df2 = df2[1:] 
df2.columns = new_header
# df2

In [43]:
df2.iloc[~pd.isnull(df2['usage>usage.material>staminal.tube'].values)]

3,organs>trait>trait_code>detail,Family,UICN,Commentaire,detail,uicn>statut_uicn>nan,uicn>statut_uicn>nan,uicn>statut_uicn>nan,uicn>statut_uicn>nan,uicn>statut_uicn>nan,...,usage>seed.poisonous>staminal.tube,usage>seed.poisonous>staminal.tube,usage>agro-forestry>staminal.tube,usage>agro-forestry>staminal.tube,usage>usage.dyeing>staminal.tube,usage>usage.dyeing>staminal.tube,usage>usage.material>staminal.tube,usage>usage.material>staminal.tube,usage>clothing>staminal.tube,usage>clothing>staminal.tube
15,Anthonotha crassifolia,Fabaceae,NaN,"pas de données dans prota, no image sur http:/...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN


Prediction(without reweighting)
the family don't share that much info, and the result is poor due to insufficient data.

In [154]:
df_w_label = df2.dropna(subset='Family')
df_w_label.iloc[:,5:] = [[df_w_label.iloc[i,j] if pd.isnull(df_w_label.iloc[i,j]) else 1 for j in range(5,n)] for i in range(len(df_w_label))]
X = df_w_label.iloc[:,5:]
y = df_w_label['Family']

# from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
m = RandomForestClassifier(n_jobs=4, random_state=3)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss


X_train,  X_test,y_train, y_test = train_test_split(X, y, test_size=.5, random_state=3)# stratify=y)
# print(X_train)
# print(y_train)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)
y_prob = m.predict_proba(X_test)
y_prob ={m.classes_[i]:[y_prob[j][i]for j in range(len(y_prob))] for i in range(len(m.classes_))} 


perf = accuracy_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
# log_loss(y_test, y_prob)
perf,# conf



(0.625,)

In [155]:
y_prob

{'APTANDRACEAE (Olacaceae)': [0.11, 0.11, 0.12, 0.12, 0.12, 0.12, 0.11, 0.11],
 'Annonaceae': [0.25, 0.24, 0.25, 0.24, 0.26, 0.24, 0.25, 0.25],
 'Combretaceae': [0.2, 0.24, 0.18, 0.19, 0.2, 0.19, 0.22, 0.21],
 'Fabaceae': [0.28, 0.29, 0.31, 0.32, 0.27, 0.29, 0.28, 0.28],
 'Rutaceae': [0.16, 0.12, 0.14, 0.13, 0.15, 0.16, 0.14, 0.15]}

In [143]:
m.classes_

array(['APTANDRACEAE (Olacaceae)', 'Annonaceae', 'Combretaceae',
       'Fabaceae', 'Rutaceae'], dtype=object)